In [173]:
version = 7

In [174]:
import json

f_raw1 = "round2.jsonl"
f_raw2 = "batch-1.jsonl"

with open(f_raw1) as fin:
    data1 = [json.loads(line) for line in fin]

with open(f_raw2) as fin:
    data2 = [json.loads(line) for line in fin]

data = data1 + data2
print(len(data))

1226


In [175]:
outputs = []
err_cnt_dist = dict()
err_type_dist = dict()
for idx,line in enumerate(data):
    # print(idx)
    
    flag = False
    if "CLASSIFICATION_JOB_2" in line["latestLabel"]["jsonResponse"]:
        flags = line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"]
        for i in range(len(flags)):
            if line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"][i]["name"] == "ERRORS_IN_THE_CORRECT_OUTPUT":
                flag = True
            elif line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"][i]["name"] == "THE_CONTEXT_IS_VERY_COMPLEX_IM_NOT_SURE_OF_MY_ANSWER":
                flag = True
    if flag:
        continue

    labels = line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_0"]["categories"]
    feedback = []
    error_types = []
    for label in labels:
        feedback.extend([i["text"] for i in label["children"].values()])
        error_types.append(label["categoryName"])
        
    # priorities
    priorities = ['Coherence and Deduction', 'Veracity', 'Arithmetic', 'Commonsense'] #'Consistency with Context', 'Redundancy']
    
    if len(feedback) > 1:
        new_feedback = []
        new_error_type = []
        
        for priority in priorities:
            for type_idx in range(len(feedback)):
                if error_types[type_idx] == priority:
                    new_error_type.append(priority)
                    new_feedback.append(feedback[type_idx])
        
        feedback = new_feedback
        error_types = new_error_type
        
    if len(feedback) < 1: continue
                    
    
    # remove candidate
    for item_id in range(len(feedback)):
        if "victory is an election result in which the victorious candidate" in feedback[item_id]: 
            feedback[item_id] = feedback[item_id].replace("Candidates", "The answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("The candidate output", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate output'answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate'answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("the candidate had the answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidates answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate does not answer the question", "this does not answer the question")
        
        feedback[item_id] = feedback[item_id].replace("The candidates", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate's answer", "The answer")
        
        feedback[item_id] = feedback[item_id].replace("The output candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("the output candidate", "the answer")
        
        feedback[item_id] = feedback[item_id].replace("candidate's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("Candidate output", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidate Output", "The answer")
        feedback[item_id] = feedback[item_id].replace("candidate answers", "it answers")
        feedback[item_id] = feedback[item_id].replace("Th candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Output Candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Output candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidates", "The answer")
        
        feedback[item_id] = feedback[item_id].replace("candidate", "the answer")
        
        
        feedback[item_id] = feedback[item_id].replace("The answer answers ", "The answer says ")
        feedback[item_id] = feedback[item_id].replace("the answer answers ", "the answer says ")
        feedback[item_id] = feedback[item_id].replace("The answer answered ", "The answer sayed ")
        feedback[item_id] = feedback[item_id].replace("the answer answered ", "the answer sayed ")
        feedback[item_id] = feedback[item_id].replace("The answer answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("The answer in the answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("the answers' answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("The answer's answer", "The answer")
        
        
        answer_loc_1 = feedback[item_id].find("answer")
        answer_loc_2 = feedback[item_id][answer_loc_1+5:].find("answer")
        if answer_loc_2 > -1 and answer_loc_2 < 10:
            print(feedback[item_id])
            print(" ")
    
    my_key = "candidate"
    for item in feedback:
        loc = -1
        if my_key in item:
            loc = item.find(my_key)
        
        if loc > -1:
            print(item[loc:loc+155])
    
    
        
    if len(feedback) in err_cnt_dist:
        err_cnt_dist[len(feedback)] += 1
    else:
        err_cnt_dist[len(feedback)] = 1
        
    for err_type in error_types:
        if err_type in err_type_dist:
            err_type_dist[err_type] += 1
        else:
            err_type_dist[err_type] = 1
        
    question = line["metadata"]["context"]
    answer = line["metadata"]["output_candidate"]
    
    critique = " ".join(feedback)
    if "correct output" in critique or "Correct output" in critique or "Correct Output" in critique:
#         print("prompt_id:", line["metadata"]["prompt_id"])
        continue
    
    outputs.append({"question": question,
                  "answer": answer,
                  "feedback": feedback,
                  "error_types": error_types})
print(len(outputs))
print(err_cnt_dist)
print(err_type_dist)

After giving the right answer, the answer goes ahead to give extra unnecessary explanation, Jason Jones was born in 1971 which the 70s and not 60s.
 
After giving the correct answer, the answer goes ahead to give explanations that are not needed to support our answer, boredom will make one to keep driving.
 
After giving the right answer, the answer explains additional information about comedy performance, preferred atmosphere which we do not need, normally we would eat first the go to a club so we would go to a restaurant.
 
After giving the wrong answer, the answer went ahead and provided unnecessary information, schools have changing rooms and gyms.
 
After giving the correct answer, the answer went ahead to explain unnecessary information there after.
 
The answer never answers the question and instead list facts of the Mississippi River about where it flows and the people to first explore the Mississippi River which isn't relevant to what's needed to select the correct choice. The

In [186]:
version = 3

processed_outputs = []
err_type_dist = dict()

bad_count_redundancy = 10
bad_count_consistent = 10

for idx, output in enumerate(outputs):
    
    if len(output["feedback"]) == 1:
        if output["error_types"][0] == "No error":
            critique = output["feedback"][0]
        elif " correct" in output["feedback"][0]:
            critique = output["feedback"][0]
        elif output["error_types"][0] in ["Redundancy", "Consistency with Context"]:
            critique = output["feedback"][0]
        else:
            critique = "" + output["feedback"][0]
            
        if output["error_types"][0] == "Redundancy":
            if bad_count_redundancy > 0:
                bad_count_redundancy -= 1
            else:
                continue
        if output["error_types"][0] == "Consistency with Context":
            if bad_count_consistent > 0:
                bad_count_consistent -= 1
            else:
                continue
            
    elif len(output["feedback"]) == 2:
        critique = "Firstly, " + output["feedback"][0] + " Secondly, " + output["feedback"][1]
    elif len(output["feedback"]) == 3:
        critique = "Firstly, " + output["feedback"][0] + " Secondly, " + output["feedback"][1] + \
        " Thirdly, " + output["feedback"][2]
    
    if "in which the victorious candidate or party wins by an overwhelming margin. Therefore, the vote could not" in critique: continue
    
    for err_type in output["error_types"]:
        if err_type in err_type_dist:
            err_type_dist[err_type] += 1
        else:
            err_type_dist[err_type] = 1
    
#     if "Veracity" in output["error_types"]:
#         print(critique)
    if idx % 70 == 0:
        print(critique)
        print(output["error_types"])
        print()
    processed_outputs.append({"id":idx,"question":output["question"],"answer":output["answer"],
                "feedback":critique,"meta":{"domain":"human annotation"}})
    



print(len(processed_outputs))

with open(f"/private/home/tianluwang/xlformers/critiqueLM/data/human/v1/v1.{version}.jsonl", "w") as fout:
    for output in processed_outputs:
        fout.write(f"{json.dumps(output)}\n")

Firstly, The math is off in the previous to last sentence, since the ages of the children should be subtracted, not summed. 15 - 1 - 4 -2 = 8 is the right answer. Secondly, "So the fourth child is 15 + 1 + 4 + 2 = 22 years old. The answer is 22." The fourth born child cannot be older than the first born child.  
['Coherence and Deduction', 'Commonsense']

The answer is correct.
['No error']

The answer is correct.
['No error']

A labyrinth is not made of corn structures so the answer's logic that a corn structure with lots of passage ways is a labyrinth is false. The correct answer would be: A maze is made up of corn plantations with many passageways and a corn maze is the most common in occurrence. Corn structures do not exist in a jungle, nor house, nor labyrinth, nor hotel. 
['Commonsense']

The premise states the ladies are out at a bar but the answer ignores this and says we don't know where they are when they are enjoying a laugh. This is false. 
['Commonsense']

The answer is co

In [187]:
print(err_type_dist)

{'Coherence and Deduction': 415, 'Commonsense': 171, 'Arithmetic': 95, 'Consistency with Context': 10, 'No error': 140, 'Redundancy': 10, 'Veracity': 78}
